# Notebook de Creación del Servicio de API para Despliegue en BentoML

# 1. Importación de librerías necesarias
Aquí simplemente traemos las librerías que necesitamos: BentoML para crear el servicio, NumPy para manejar los números y Pydantic para definir cómo son los datos que vamos a recibir.

In [1]:
import bentoml
import numpy as np
from pydantic import BaseModel, Field

# 2. Cargar el modelo entrenado desde el Model Store

En este bloque, definimos los "nombres clave" (Tags) con los que guardamos nuestros modelos y escaladores anteriormente.

In [2]:
XGB_TAG            = "ai4i2020_xgbclassifier:latest"
LOGR_TAG           = "ai4i2020_logistic_regression:latest"
SVM_TAG            = "ai4i2020_support_vector_machine:latest"
RF_TAG             = "ai4i2020_random_forest:latest"
HDBSCAN_MODEL_TAG  = "ai4i2020_hdbscan:latest"

XGB_SCALER_TAG       = "ai4i2020_scaler_xgbclassifier:latest"
LOGR_SCALER_TAG      = "ai4i2020_scaler_logistic_regression:latest"
SVM_SCALER_TAG       = "ai4i2020_scaler_svm:latest"
RF_SCALER_TAG        = "ai4i2020_scaler_random_forest:latest"
HDBSCAN_SCALER_TAG   = "ai4i2020_scaler_hdbscan:latest"

# 3. Definir los Schemas de entrada

Aquí creamos dos moldes para los datos. Además, ponemos un ejemplo por defecto para que aparezca bonito en la página web del servicio.

In [3]:
# Esquema para modelos de 12 columnas (Logistic, SVM, XGB)
class Input12Features(BaseModel):
    # Definimos que esperamos una lista de listas de floats
    input_data: list[list[float]] = Field(
        default=[[298.9, 309.1, 2861, 4.6, 143, 0, 0, 1, 0, 0, 1, 0]],
        description="Matriz de entrada con 12 características."
    )

# Esquema para modelos de 7 columnas (Random Forest, HDBSCAN)
class Input7Features(BaseModel):
    input_data: list[list[float]] = Field(
        default=[[298.8, 308.9, 1455, 41.3, 208, 1, 0]],
        description="Matriz de entrada con 7 características."
    )

# 4. Cargamos los modelos en la memoria
Este es el momento en que el servidor arranca. Usamos las etiquetas que definimos antes para sacar los modelos reales del disco y cargarlos en la memoria.

In [4]:
print("Cargando MODELOS del Model Store...")
xgb_model   = bentoml.sklearn.load_model(XGB_TAG)
logr_model  = bentoml.sklearn.load_model(LOGR_TAG)
svm_model   = bentoml.sklearn.load_model(SVM_TAG)
rf_model    = bentoml.sklearn.load_model(RF_TAG)
hdb_model   = bentoml.sklearn.load_model(HDBSCAN_MODEL_TAG)

print("Cargando SCALERS del Model Store...")
xgb_scaler   = bentoml.picklable_model.load_model(XGB_SCALER_TAG)
logr_scaler  = bentoml.picklable_model.load_model(LOGR_SCALER_TAG)
svm_scaler   = bentoml.picklable_model.load_model(SVM_SCALER_TAG)
rf_scaler    = bentoml.picklable_model.load_model(RF_SCALER_TAG)
hdb_scaler   = bentoml.picklable_model.load_model(HDBSCAN_SCALER_TAG)

print("Modelos y scalers cargados.")

Cargando MODELOS del Model Store...


c:\Users\diego\Carrera\4º\Analítica\ADDPLI---Proyecto-Final\analitica\Lib\site-packages\fs\__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)  # type: ignore


Cargando SCALERS del Model Store...
Modelos y scalers cargados.


C:\Users\diego\AppData\Local\Temp\ipykernel_1440\2873539173.py:9: BentoMLDeprecationWarning: `bentoml.picklable_model` is deprecated since v1.4 and will be removed in a future version.
  xgb_scaler   = bentoml.picklable_model.load_model(XGB_SCALER_TAG)


# 5. Configuramos el servicio y nuestro asistente de limpieza
Aquí empieza la clase principal. Primero, creamos una función "ayudante" (_prepare_data) que usamos internamente. Esta función se encarga de recibir los datos brutos, convertirlos al formato NumPy y comprobar que el tamaño sea el correcto antes de pasárselos a los modelos.

In [5]:
@bentoml.service(name="AI4I2020__Failure__Prediction__Service")
class AI4I2020FailurePredictionService:
    """
    Servicio BentoML con Inputs definidos mediante Pydantic
    para mostrar ejemplos en la UI (Swagger).
    """

    # Convertir pydantic a numpy y validar
    def _prepare_data(self, pydantic_input, expected_cols: int) -> np.ndarray:
        # Extraemos la lista del objeto pydantic y convertimos a numpy
        data = np.array(pydantic_input.input_data)
        
        # Aseguramos 2D
        if data.ndim == 1:
            data = data.reshape(1, -1)
            
        # Validamos columnas
        if data.shape[1] != expected_cols:
            raise ValueError(f"Se esperaban {expected_cols} columnas, recibidas {data.shape[1]}.")
            
        return data

# 6. Creamos las "ventanillas" de atención (Endpoints)
Finalmente, definimos las funciones que estarán disponibles para el público. Cada función hace lo mismo:
1. Recibe los datos del usuario.
2. Llama al ayudante (_prepare_data) para limpiarlos.
3. Escala los datos.
4. Le pide al modelo correspondiente una predicción y nos la devuelve.

In [6]:
# Logistic Regression (Usa Input12Features)
@bentoml.api
def predict_logreg(self, input_obj: Input12Features) -> np.ndarray:
    data = self._prepare_data(input_obj, 12)
    scaled = logr_scaler.transform(data)
    return logr_model.predict_proba(scaled)

# Random Forest (Usa Input7Features)
@bentoml.api
def predict_random_forest(self, input_obj: Input7Features) -> np.ndarray:
    data = self._prepare_data(input_obj, 7)
    scaled = rf_scaler.transform(data)
    return rf_model.predict(scaled)

# SVM (Usa Input12Features)
@bentoml.api
def predict_svm(self, input_obj: Input12Features) -> np.ndarray:
    data = self._prepare_data(input_obj, 12)
    scaled = svm_scaler.transform(data)
    return svm_model.predict_proba(scaled)

# XGBoost (Usa Input12Features)
@bentoml.api
def predict_xgb(self, input_obj: Input12Features) -> np.ndarray:
    data = self._prepare_data(input_obj, 12)
    scaled = xgb_scaler.transform(data)
    return xgb_model.predict(scaled)

# HDBSCAN (Usa Input7Features)
@bentoml.api
def cluster_hdbscan(self, input_obj: Input7Features) -> np.ndarray:
    data = self._prepare_data(input_obj, 7)
    scaled = hdb_scaler.transform(data)
    # Recordamos que aquí estamos usando el truco del KNN wrapper
    # por lo que podemos usar .predict() directamente
    predictions = hdb_model.predict(scaled)
    return predictions

In [7]:
!bentoml serve service:AI4I2020FailurePredictionService --port 3000

^C


http://localhost:3000/